**Задание**
1.  Дообучить берт на задачу NER;
2.  Дообучить GPT на генерацию текста;
3*. Дообучить T5 на задачу суммаризации текста.

## 2. GPT

In [1]:
!pip install -q transformers

In [2]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

## Loading Data

https://www.kaggle.com/coolonce/recipes-and-interpretation-dim

In [3]:
df = pd.read_csv('all_recepies_inter.csv', sep='\t')

In [4]:
df.head(3)

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."


In [5]:
df.shape

(27884, 10)

In [6]:
print(df.loc[0, 'Инструкции'])

Подготовить указанные ингредиенты для приготовления рассольника с перловой крупой. Мясной бульон сварить заранее из говядины или из курицы, также можно сварить и вегетарианский суп - на воде. Обычно я рассольник варю без томатной пасты, но тут для разнообразия решила добавить - это по желанию. Из специй соль, чёрный перец горошком, душистый перец.
Перловую крупу промыть до чистой воды.
В горячий бульон добавить промытую перловку и варить на среднем огне.
Для рассольника лучше брать кислые, очень солёные огурцы. Если же огурцы обычные, то рекомендуется в рассольник добавлять из сам рассол от огурцов. Солёные огурцы достать из рассола и натереть на крупной тёрке.
Картофель помыть, обсушить, очистить. Нарезать кубиками. Пока очередь картофеля не подошла, положить его в воду.
Морковь, лук, чеснок очистить. Морковь натереть на крупной тёрке, лук, сельдерей, чеснок мелко порезать.
Обжарить в масле овощи, добавив томатной пасты. Томатная паста по желанию.
Минут через двадцать добавить к перло

In [7]:
data = df.loc[:5000, 'Инструкции']

In [8]:
data.shape

(5001,)

In [9]:
def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
        summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [10]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train, 'train_dataset.txt')
build_text_files(test, 'test_dataset.txt')

In [11]:
print("Train dataset length:", len(train))
print("Test dataset length:", len(test))

Train dataset length: 4250
Test dataset length: 751


In [12]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

In [15]:
# !pip install accelerate -U

In [16]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500, # number of warmup steps for learning rate scheduler
    )

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [18]:
trainer.train()

Step,Training Loss
500,2.531600
1000,2.454400
1500,2.369100
2000,2.231500
2500,2.154600
3000,2.143400
3500,2.091600
4000,1.977400
4500,1.980800
5000,1.959300


TrainOutput(global_step=5007, training_loss=2.1890662709276714, metrics={'train_runtime': 1047.5001, 'train_samples_per_second': 19.12, 'train_steps_per_second': 4.78, 'total_flos': 1308289204224000.0, 'train_loss': 2.1890662709276714, 'epoch': 3.0})

In [19]:
trainer.save_model()

In [20]:
tokenizer.save_pretrained('gpt_chf')
model.save_pretrained('model_gpt_chf')

In [21]:
tokenizer = AutoTokenizer.from_pretrained("gpt_chf")
model1 = AutoModelForCausalLM.from_pretrained("model_gpt_chf")

In [34]:
prefix = "Масло и лук"

In [35]:
tokens = tokenizer(prefix, return_tensors='pt')

In [36]:
size = tokens['input_ids'].shape[1]
output = model1.generate(
    **tokens,
    #end_token=end_token_id,
    do_sample=False,
    max_length=size+50,
    repetition_penalty=5.,
    temperature=0.5,
    num_beams=10,
)

decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(prefix + result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Масло и лук обжарить на растительном масле до золотистого цвета.  3. Влить в сковороду куриный бульон, довести до кипения, убавить огонь так, чтобы вода едва кипела, накрыть крышкой и тушить 15–20
